In [ ]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import xgboost as xgb

pd.set_option('display.float_format',lambda x: '%.5f'%x)

print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path)

dfPred= pd.read_csv(config.original_test_data_path)
dfPred2= pd.read_csv(config.original_test_data2_path)
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close() 

dfTrain = dfTrain.rename(columns=col_name)
raw_predictors = dfTrain.columns.tolist()[1:-1]
dfPred = dfPred.rename(columns=col_name)
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

#dfAll = dfAll.replace({'':np.nan,0:np.nan})
dfAll = dfAll.reset_index()

In [ ]:
dfAll = pd.read_csv('../../Data/All_non_missing_v2.csv')

In [ ]:
dfAll[pro_feat] = dfAll[pro_feat]-dfAll[pro_feat].mean()

In [ ]:
#dfAll = dfAll.replace({-99999999:-2.2222222})
dfAll[['750X1450','750X1451']]

In [ ]:
#dfAll = dfAll.replace({np.nan:-99999999})
pro_feat = [var for var in raw_predictors if not 'TOOL' in var ]
corr = dfAll[pro_feat].corr()
for i in range(len(corr)):
    corr.iloc[i,i:] =np.nan
print((corr==1.0).sum(axis=1))

In [ ]:
corr

In [ ]:
#处理0这个比较异常的值,在trian set中有40条数据，在某些column上为0 ，需要处理，test_a中也有一条
feature = ['ID']
for var in raw_predictors:
    if 'TOOL' in var:
        feature.append(var)
pick_list = (((corr==1.0).sum(axis=1))==0).values
for i in range(len(pick_list)):
    if pick_list[i]==True:
        feature.append(corr.index[i])
print(len(feature))

In [ ]:
#number of NAN
dfAll['Total_NAN_Num']=dfAll[raw_predictors].T.isnull().sum().tolist()
for key,pro in procedure.items():
    dfAll[key+'_NAN_Num']=dfAll[pro].T.isnull().sum().tolist()

In [ ]:
raw_predictors = dfAll.columns.tolist()[1:-13]
raw_predictors[-1]

In [ ]:
#Count 计算特定值在变量中重复出现的次数'''
dfAll = dfAll.replace({np.nan:-99999999})
added = ['ID']
for var in raw_predictors:
    if not var in dfAll.columns:
        continue
    dfAll['cnt_'+var] = 0
    added.append('cnt_'+var)
    groups = dfAll.groupby([var])
    for name,group in groups:
        count = group[var].count()
        dfAll['cnt_'+var].ix[group.index] = count
dfAll[added].to_csv('../../Cache/Feat_cnt_col_nonmissing.csv',index=False)

added = ['ID']
for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]])
    for var in pro:
        if not var in dfAll.columns:
            continue
        dfAll['cnt_'+tool+'_'+var] = 0
        added.append('cnt_'+tool+'_'+var)
        for name,group in groups:
            grps = group.groupby([var])
            for name2,grp in grps:
                dfAll['cnt_'+tool+'_'+var].ix[grp.index] = float(len(grp))/float(len(group))
dfAll[added].to_csv('../../Cache/Feat_cnt_tool_nonmissing.csv',index=False)

In [ ]:
#percentile 计算某一个值在全集中的位置（百分制）
dfAll = dfAll.replace({-99999999:np.nan})
added = ['ID']
for var in raw_predictors:
    if not var in dfAll.columns:
        continue
    if 'TOOL' not in var:
        dfAll['pcent_'+var] = dfAll[var].rank(method='max')/float(len(dfAll))
        added.append('pcent_'+var)
dfAll[added].to_csv('../../Cache/Feat_pcent_col_nonmissing.csv',index=False)
t = 0
added = ['ID']
for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]]) 
    for var in pro:
        if not var in dfAll.columns:
            continue
        dfAll['pcent_'+tool+'_'+var] = 0
        added.append('pcent_'+tool+'_'+var)
        for name,group in groups:
            dfAll['pcent_'+tool+'_'+var].ix[group.index] = group[var].rank(method='max')/float(len(group))
            
dfAll[added].to_csv('../../Cache/Feat_pcent_tool_nonmissing.csv',index=False) 

In [ ]:
len(raw_predictors)

In [ ]:
minus_var = ['ID']
for i in range(len(raw_predictors)-1):
    for j in range(i+1,len(raw_predictors)):
        if (dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].max())*(dfAll[raw_predictors[i]].min()-dfAll[raw_predictors[j]].min())<0:
            if float(min(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[i]].min(),dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[j]].min()))/max(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[i]].min(),dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[j]].min())>=0.6:
                tmp_var = raw_predictors[i]+'-'+raw_predictors[j]
                dfAll[tmp_var] = dfAll[raw_predictors[i]]-dfAll[raw_predictors[j]]
                minus_var.append(tmp_var)
                print(tmp_var)
        elif (dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min())*(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min())>0:
            if float(min(abs(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min()),abs(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min())))/max(abs(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min()),abs(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min()))>=0.6:
                tmp_var = raw_predictors[i]+'-'+raw_predictors[j]
                dfAll[tmp_var] = dfAll[raw_predictors[i]]-dfAll[raw_predictors[j]]
                minus_var.append(tmp_var)
                print(tmp_var)

In [ ]:
dffAll_minus = dfAll[minus_var]
dffAll_minus.to_csv('../../../All_minus.csv',index=False)

In [ ]:
(dffAll_minus.nunique()<=10).sum()

In [ ]:
added = ['ID']
for var in minus_var[1:]:
    if dffAll_minus[var].nunique()>10:
        continue
    dffAll_minus['cnt_'+var] = 0
    added.append('cnt_'+var)
    groups = dffAll_minus.groupby([var])
    for name,group in groups:
        count = group[var].count()
        dffAll_minus['cnt_'+var].ix[group.index] = count
for var in minus_var[1:]:
    if 'TOOL' not in var:
        dfAll['pcent_'+var] = dfAll[var].rank(method='max')/float(len(dfAll))
        added.append('pcent_'+var)

In [ ]:
#Pivot 创建多组category variable 的两两交叉表,感觉只能用来计算分类型变量,好像现在直接可以用pandas解决
def get_pivottable(X_train, X_test, use='all',Feat_list):
    """
    Returns a list of dictionaries, one per feature in the
    basic data, containing cross-tabulated counts
    for each column and each value of the feature.
    """
    dictionaries = []
    if use == 'all':
        X = np.vstack((X_train, X_test))
        filename = "pivottable"
    elif use == 'train':
        X = X_train
        filename = "pivottable_train"
    else:
        X = X_test
        filename = "pivottable_test"
    for i in range(len(Feat_list)):
        dictionaries.append({'total': 0})
    try:
        with open("cache/%s.pkl" % filename, 'rb') as f:
            logger.debug("loading cross-tabulated data from cache")
            dictionaries = pickle.load(f)
    except IOError:
        logger.debug("no cache found, cross-tabulating data")
        for i, row in enumerate(X):
            for j in range(len(Feat_list)):
                dictionaries[j]['total'] += 1
                if row[j] not in dictionaries[j]:
                    dictionaries[j][row[j]] = {'total': 1}
                    for k, key in enumerate(Feat_list):
                        dictionaries[j][row[j]][key] = {row[k]: 1}
                else:
                    dictionaries[j][row[j]]['total'] += 1
                    for k, key in enumerate(Feat_list):
                        if row[k] not in dictionaries[j][row[j]][key]:
                            dictionaries[j][row[j]][key][row[k]] = 1
                        else:
                            dictionaries[j][row[j]][key][row[k]] += 1
        with open("cache/%s.pkl" % filename, 'wb') as f:
            pickle.dump(dictionaries, f, pickle.HIGHEST_PROTOCOL)

    return dictionaries

def creat_cate_corr(X_train, X_test, Feat_list=[var if 'TOOL' in var for var in raw_predictor]):
    features_train = []
    features_test = []
    dictionaries = get_pivottable(X_train[Feat_list], X_test[Feat_list],Feat_list=Feat_list)
    #dictionaries_train = get_pivottable(X_train[Feat_list], X_test[Feat_list], use='train',Feat_list=Feat_list)
    #dictionaries_test = get_pivottable(X_test[Feat_list], X_test[Feat_list], use='test',Feat_list=Feat_list)

In [ ]:
dfAll = pd.read_csv('../../Data/All_Data_dedup.csv')
feat_needed = dfAll.columns.tolist()[14:]
'''gap_list = []
for i in range(len(feat_needed)-1):
    for j in range(i+1,len(feat_needed)):
        if dfAll[feat_needed[i]].quantile(0.5) - dfAll[feat_needed[j]].quantile(0.5)<=1 and dfAll[feat_needed[i]].quantile(0.5) - dfAll[feat_needed[j]].quantile(0.5)>=-1:
            tmp_var = feat_needed[i]+'-'+feat_needed[j]
            dfAll[tmp_var] = dfAll[feat_needed[i]]- dfAll[feat_needed[j]]
            gap_list.append(tmp_var)
            print(tmp_var)'''

In [ ]:
feat_needed2 =[]
for var in feat_needed:
    if dfAll[var].nunique()>15:
        feat_needed2.append(var)
gap_list = []
for i in range(len(feat_needed2)-1):
    for j in range(i+1,len(feat_needed2)):
        if dfAll[feat_needed2[i]].quantile(0.5) - dfAll[feat_needed2[j]].quantile(0.5)<=2 and dfAll[feat_needed2[i]].quantile(0.5) - dfAll[feat_needed2[j]].quantile(0.5)>=-2:
            tmp_var = feat_needed2[i]+'-'+feat_needed2[j]
            dfAll[tmp_var] = dfAll[feat_needed2[i]]- dfAll[feat_needed2[j]]
            gap_list.append(tmp_var)
            print(tmp_var)

In [ ]:
dfAll = dfAll.set_index(dfAll['ID'])
dfTrain_tmp = dfAll.loc[dfTrain['ID']]
dfPred_tmp = dfAll.loc[dfPred['ID']]

In [ ]:
dfPred = dfPred_tmp.copy(deep=True)
dfTrain = dfTrain_tmp.copy(deep=True)

In [ ]:
Feat_file = ['Feat_cnt_col','Feat_cnt_tool','Feat_pcent_col','Feat_pcent_tool']
ex_feat = []
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')

In [ ]:
dfAll2 = pd.read_csv('../../Data/All_Data_dedup.csv',nrows=2)
raw_predictors = dfAll2.columns[1:]

In [ ]:
dfTrain.columns

In [ ]:
f = open('../../Cache/var_change.txt','r')
a = f.read()
var_change = eval(a)
f.close()
predictors = dfTrain.columns.tolist()[1:]+ex_feat
feat_cnt = 0
for key,value in var_change.items():
    feat_cnt+=1
    if feat_cnt>1:
        continue
    for var in var_change[key]['constant']:
        try:
             predictors.remove(var)
        except:
            continue
    for var in var_change[key]['category']:
        try:
            predictors.remove(var)
        except:
            continue
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        dfPred[var] = 0

In [ ]:
def xgb_kfold(dfTrain,dfPred,predictors,n_splits=5, params = {'max_depth':3, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1,'subsample':0.8,
                         'colsample_bytree':0.4}):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors]
        test_X = dfTrain.loc[test_index,predictors]
        train_Y = dfTrain.loc[train_index,'Y']
        test_Y = dfTrain.loc[test_index,'Y']

        dtrain = xgb.DMatrix(train_X.values, label=train_Y.values, missing = np.nan)
        dtest = xgb.DMatrix(test_X.values, label=test_Y.values, missing = np.nan)
        param = params 
        evallist  = [(dtrain,'train'),(dtest,'eval')]  
        num_round = 5000
        evals_dict = {}
        model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=50,evals_result=evals_dict,verbose_eval =1000)
        performance_df = pd.DataFrame(evals_dict['eval'])
        bst_tree = len(performance_df)-51
        pred_test = model.predict(dtest,ntree_limit =bst_tree)

        tmp_imp = pd.DataFrame(model.get_score(importance_type='gain'),index=['imp_fold%d'%round]).T
        tmp_imp['lk'] = tmp_imp.index
        imp = imp.merge(tmp_imp,'left','lk').fillna(0)


        pred_score = model.predict(dpred,ntree_limit =bst_tree)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    #print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp

In [ ]:
dfTrain2 = pd.read_csv(config.original_train_data_path,usecols=['ID','Y'])
dfTrain = dfTrain.merge(dfTrain2,'inner','ID')
dfTrain.shape

In [ ]:
test_result,result,imp = xgb_kfold(dfTrain,dfPred,predictors)

In [ ]:
result['score']=result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit_%s.csv'%today,header=False,index=False)
imp.to_csv('../../Profiling/imp_test_v3_%s.csv'%today,index=False)
test_result('../../Profiling/test_score_%s.csv'%today,index=False)